In [ ]:
# Program:  Calculates hedging a physical inventory position using futures in either backwardated or contango market
# author: Luis A. Molina
# Date:  2024-10-09

#install necessary packages
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid')

# Define the FuturesCurve class
class FuturesCurve:
    def __init__(self, start_date, end_date, initial_spot_price, market_state='contango'):
        self.start_date = start_date
        self.end_date = end_date
        self.initial_spot_price = initial_spot_price
        self.market_state = market_state
        self.dates = pd.date_range(start_date, end_date, freq='D')
        self.num_periods = len(self.dates)
        self.spot_prices = self.generate_spot_prices()
        self.futures_prices = self.generate_futures_prices()
    
    def generate_spot_prices(self):
        # Simulate spot prices over time using a random walk
        np.random.seed(42)  # Set seed for reproducibility
        spot_prices = [self.initial_spot_price]
        for _ in range(1, self.num_periods):
            change = np.random.normal(0, 0.2)  # Mean zero, standard deviation of 0.2
            new_price = spot_prices[-1] + change
            spot_prices.append(new_price)
        return spot_prices
    
    def generate_futures_prices(self):
        # Generate futures prices that converge to the spot price
        futures_prices = []
        total_time_to_maturity = self.num_periods - 1
        for i in range(self.num_periods):
            time_to_maturity = total_time_to_maturity - i
            if self.market_state == 'contango':
                # Futures price starts above spot and converges downward
                futures_price = self.spot_prices[i] + time_to_maturity * 0.10  # 10 cents per day carry cost
            elif self.market_state == 'backwardation':
                # Futures price starts below spot and converges upward
                futures_price = self.spot_prices[i] - time_to_maturity * 0.10  # 10 cents per day discount
            else:
                raise ValueError("Invalid market state. Choose 'contango' or 'backwardation'.")
            futures_prices.append(futures_price)
        return futures_prices


# Define the PhysicalPosition class
class PhysicalPosition:
    def __init__(self, quantity, purchase_price, storage_cost_per_day):
        self.quantity = quantity  # Positive for long position
        self.purchase_price = purchase_price
        self.storage_cost_per_day = storage_cost_per_day
        self.position = quantity
        self.total_storage_cost = 0.0
        self.pnl = 0.0
    
    def update_storage_cost(self):
        self.total_storage_cost += self.storage_cost_per_day * self.quantity
    
    def sell(self, sale_price):
        self.pnl = (sale_price - self.purchase_price) * self.quantity

# Define the HedgingStrategy class
class HedgingStrategy:
    def __init__(self, futures_curve, physical_position):
        self.futures_curve = futures_curve
        self.physical_position = physical_position
        self.dates = futures_curve.dates
        self.results = pd.DataFrame(index=self.dates)
        self.results['Spot Price'] = futures_curve.spot_prices
        self.results['Futures Price'] = futures_curve.futures_prices
        self.results['Physical Position'] = physical_position.quantity
        self.results['Futures Position'] = -physical_position.quantity  # Short futures to hedge
        self.results['Physical P&L'] = 0.0
        self.results['Futures P&L'] = 0.0
        self.results['Storage Cost'] = 0.0
        self.results['Net P&L'] = 0.0
        self.results['Cumulative P&L'] = 0.0
        self.results['Position'] = physical_position.quantity  # For plotting positions over time
    
    def simulate(self):
        cumulative_pnl = 0.0
        previous_futures_price = self.results.iloc[0]['Futures Price']
        for i, date in enumerate(self.dates):
            # Update storage cost
            if i > 0:
                self.physical_position.update_storage_cost()
            self.results.at[date, 'Storage Cost'] = -self.physical_position.total_storage_cost
            
            # Update physical P&L
            if i == len(self.dates) - 1:
                # Sell the physical position at spot price
                sale_price = self.results.at[date, 'Spot Price']
                self.physical_position.sell(sale_price)
                self.results.at[date, 'Physical P&L'] = self.physical_position.pnl
                self.results.at[date, 'Physical Position'] = 0
                self.results.at[date, 'Futures Position'] = 0
                self.results.at[date, 'Position'] = 0  # Update position to zero after selling
            else:
                self.results.at[date, 'Physical P&L'] = 0.0  # Unrealized P&L not counted
            
            # Update futures P&L
            current_futures_price = self.results.at[date, 'Futures Price']
            if i > 0:
                # Realized P&L from closing previous futures position
                futures_pnl = (previous_futures_price - current_futures_price) * self.physical_position.quantity
                self.results.at[date, 'Futures P&L'] = futures_pnl
            else:
                futures_pnl = 0.0
            previous_futures_price = current_futures_price
            
            # Calculate net P&L
            net_pnl = (self.results.at[date, 'Physical P&L'] +
                       self.results.at[date, 'Futures P&L'] +
                       self.results.at[date, 'Storage Cost'])
            cumulative_pnl += net_pnl
            self.results.at[date, 'Net P&L'] = net_pnl
            self.results.at[date, 'Cumulative P&L'] = cumulative_pnl

    def plot_results(self):
        # Extract data and convert to numpy arrays
        dates = self.results.index.to_numpy()
        spot_prices = self.results['Spot Price'].to_numpy()
        futures_prices = self.results['Futures Price'].to_numpy()
        cumulative_pnl = self.results['Cumulative P&L'].to_numpy()
        physical_position = self.results['Physical Position'].to_numpy()
        futures_position = self.results['Futures Position'].to_numpy()
        
        # Create subplots
        fig, axs = plt.subplots(3, 1, figsize=(12, 15))
        
        # Plot spot and futures prices
        axs[0].plot(dates, spot_prices, label='Spot Price')
        axs[0].plot(dates, futures_prices, label='Futures Price')
        axs[0].set_title('Spot and Futures Prices Over Time')
        axs[0].set_xlabel('Date')
        axs[0].set_ylabel('Price ($)')
        axs[0].legend()
        axs[0].grid(True)
        
        # Plot cumulative P&L
        axs[1].plot(dates, cumulative_pnl, label='Cumulative P&L', color='green')
        axs[1].set_title('Cumulative P&L Over Time')
        axs[1].set_xlabel('Date')
        axs[1].set_ylabel('P&L ($)')
        axs[1].legend()
        axs[1].grid(True)
        
        # Plot positions over time
        axs[2].plot(dates, physical_position, label='Physical Position')
        axs[2].plot(dates, futures_position, label='Futures Position')
        axs[2].set_title('Positions Over Time')
        axs[2].set_xlabel('Date')
        axs[2].set_ylabel('Quantity')
        axs[2].legend()
        axs[2].grid(True)
        
        # Adjust layout and show plot
        plt.tight_layout()
        plt.show()
        

# Main function to run the simulation
def main():
    # User input for market state
    market_state = input("Enter market state ('contango' or 'backwardation'): ").strip().lower()
    if market_state not in ['contango', 'backwardation']:
        print("Invalid market state. Defaulting to 'contango'.")
        market_state = 'contango'
    
    # Parameters
    start_date = '2024-01-01'
    end_date = '2024-03-31'  # Simulate for 3 months
    
    initial_spot_price = 75.0
    quantity = 1000  # Barrels
    storage_cost_per_day = 0.0  # Set storage cost per day to zero
    
    # Generate futures curve
    futures_curve = FuturesCurve(start_date, end_date, initial_spot_price, market_state)
    
    # Create physical position
    physical_position = PhysicalPosition(quantity, initial_spot_price, storage_cost_per_day)
    
    # Simulate hedging strategy
    hedging_strategy = HedgingStrategy(futures_curve, physical_position)
    hedging_strategy.simulate()
    
    # Display results
    pd.set_option('display.float_format', '{:.2f}'.format)
    print("\nSimulation Results:")
    print(hedging_strategy.results)
    
    # Plot results
    hedging_strategy.plot_results()
    
    # Display the final cumulative P&L
    final_pnl = hedging_strategy.results['Cumulative P&L'].iloc[-1]
    print(f"\nFinal Cumulative P&L: ${final_pnl:,.2f}")
    
if __name__ == '__main__':
    main()
